In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix
from catboost import CatBoostClassifier
import matplotlib.pyplot as plt
import catboost

def load_and_preprocess_data(filename):
    df = pd.read_csv(filename)
    df = df.sample(frac=1, random_state=7).reset_index(drop=True)  # Shuffling
    X = df.drop('target', axis=1)
    y = df['target']
    return X, y

def train_catboost(X_train, y_train, iters, lr, d, a, b):
    cb_model = CatBoostClassifier(
        iterations=iters,
        learning_rate=lr,
        depth=d,
        random_seed=7,
        bagging_temperature=1,
        logging_level='Silent',
        class_weights=[a, b],
        task_type="GPU",
        devices='0,1'
    )
    cb_model.fit(X_train, y_train)
    return cb_model

def plot_feature_importance(df, title, color):
    plt.figure(figsize=(12, 7))
    df.plot('feature', 'importance', kind='bar', color=color)
    plt.title(title, fontsize=14)
    plt.xlabel("Features")
    plt.ylabel("Importance")
    plt.show()

def model_cats_test0_imp0(iters0, lr0, d0, a0, b0, train_name0, test_name0):
    X_train, y_train = load_and_preprocess_data(train_name0)
    X_valid, y_valid = load_and_preprocess_data(test_name0)
    
    cb_model = train_catboost(X_train, y_train, iters0, lr0, d0, a0, b0)
    
    cm1 = confusion_matrix(y_valid, cb_model.predict(X_valid))
    roca = round(roc_auc_score(y_valid, cb_model.predict(X_valid)), 4)
    sens = round(cm1[1,1] / (cm1[1,1] + cm1[0,1]), 4)
    spec = round(cm1[0,0] / (cm1[1,0] + cm1[0,0]), 4)
    f1sc = round(f1_score(y_valid, cb_model.predict(X_valid)), 4)
    
    fi_l = cb_model.get_feature_importance()
    dfi = pd.DataFrame({"feature": list(X_valid.columns), "importance": fi_l})
    dfi = dfi.sort_values(by="importance", ascending=False)
    plot_feature_importance(dfi, "Feature Importance of CatBoost", 'c')
    
    fi0_l = cb_model.get_feature_importance(catboost.Pool(X_valid, label=y_valid), type="LossFunctionChange")
    dfi0 = pd.DataFrame({"feature": list(X_valid.columns), "importance": fi0_l})
    dfi0 = dfi0.sort_values(by="importance", ascending=False)
    plot_feature_importance(dfi0, "Feature Importance of LossFunctionChange", 'm')
        
    print(". --- .")
    print(" ")
    print(spec, sens, roca, f1sc)
    return dfi, dfi0

def plot_importance(df, title, color, k):
    plt.figure(figsize=(12, 7))
    df_head = df.head(k)
    ax = df_head.plot('feature', 'importance', kind='bar', color=color)
    ax.set_title(title, fontsize=14)
    plt.show()

def show_imp(di, di0, k):
    plot_importance(di, "Feature Importance of CatBoost", 'c', k)
    print(". - .")
    print(" ")
    plot_importance(di0, "Feature Importance of LossFunctionChange", 'm', k)